# Getting Started with Snowflake Openflow Kafka Connector

Companion code for the [Snowflake Openflow Kafka Connector Quickstart](https://quickstarts.snowflake.com/guide/getting_started_with_openflow_kafka_connector/index.html).

## Overview

This quickstart demonstrates how to build a real-time streaming pipeline from Apache Kafka to Snowflake using the Openflow Kafka Connector. You'll learn how to:

- Set up a Kafka topic for application log streaming
- Configure Snowflake objects (database, schema, tables, network rules)
- Deploy Openflow SPCS runtime
- Configure the Kafka connector in Openflow Canvas
- Stream real-time logs from Kafka to Snowflake
- Perform powerful SQL analytics on streaming log data


## Database Setup
The following cell sets up all required Snowflake objects for the Kafka log streaming demo. Run this BEFORE configuring the Openflow connector

In [ ]:

USE ROLE ACCOUNTADMIN;

-- Step 1: Create Role and Database
-- ----------------------------------------------------------------------------

-- Create runtime role (reuse if coming from SPCS quickstart)
CREATE ROLE IF NOT EXISTS QUICKSTART_ROLE;

-- Create database for Kafka streaming data
CREATE DATABASE IF NOT EXISTS QUICKSTART_KAFKA_CONNECTOR_DB;

-- Create warehouse for data processing and queries
CREATE WAREHOUSE IF NOT EXISTS QUICKSTART_KAFKA_CONNECTOR_WH
  WAREHOUSE_SIZE = XSMALL
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE
  INITIALLY_SUSPENDED = TRUE;

-- Grant privileges to runtime role
GRANT OWNERSHIP ON DATABASE QUICKSTART_KAFKA_CONNECTOR_DB TO ROLE QUICKSTART_ROLE;
GRANT OWNERSHIP ON SCHEMA QUICKSTART_KAFKA_CONNECTOR_DB.PUBLIC TO ROLE QUICKSTART_ROLE;
GRANT USAGE ON WAREHOUSE QUICKSTART_KAFKA_CONNECTOR_WH TO ROLE QUICKSTART_ROLE;

-- Grant runtime role to Openflow admin
GRANT ROLE QUICKSTART_ROLE TO ROLE OPENFLOW_ADMIN;

## Create Network Rule

Create network rule to allow Openflow runtime to access the Kafka clusters

> **IMPORTANT**: Replace with your Kafka broker endpoint(s)
> This quickstart works with any Kafka service:
> - GCP Managed Kafka:    `34.123.45.67:9092(public IP)`
> - AWS MSK:              `b-1.mycluster.kafka.us-east-1.amazonaws.com:9092`
> - Confluent Cloud:      `pkc-xxxxx.us-east-1.aws.confluent.cloud:9092`
> - Azure Event Hubs:     `myeventhub.servicebus.windows.net:9093`
> - Self-hosted:          `kafka.mycompany.com:9092`
>
> **NOTE**: Ensure network connectivity and firewall rules allow Snowflake access
> For multiple brokers (recommended), include all broker endpoints in VALUE_LIST

In [ ]:
USE ROLE QUICKSTART_ROLE;
USE DATABASE QUICKSTART_KAFKA_CONNECTOR_DB;

-- Create schema for network rules
CREATE SCHEMA IF NOT EXISTS QUICKSTART_KAFKA_CONNECTOR_DB.NETWORKS;

CREATE OR REPLACE NETWORK RULE QUICKSTART_KAFKA_CONNECTOR_DB.NETWORKS.kafka_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = (
    'pkc-n98pk.us-west-2.aws.confluent.cloud:9092'
    -- 'YOUR-KAFKA-BROKER-2:9092',  -- Add additional brokers if available
    -- 'YOUR-KAFKA-BROKER-3:9092'   -- Remove if you have fewer brokers
  );

In [ ]:
DESC NETWORK RULE QUICKSTART_KAFKA_CONNECTOR_DB.NETWORKS.kafka_network_rule;

## Create External Access Integration(EAI)

Create an EAI to that can be associated with the Openflow runtime

In [ ]:
USE ROLE ACCOUNTADMIN;

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION quickstart_kafka_connector_access
  ALLOWED_NETWORK_RULES = (
    QUICKSTART_KAFKA_CONNECTOR_DB.NETWORKS.kafka_network_rule
  )
  ENABLED = TRUE
  COMMENT = 'Openflow SPCS runtime access for Kafka connector';

-- Grant usage to runtime role
GRANT USAGE ON INTEGRATION quickstart_kafka_connector_access TO ROLE OPENFLOW_ADMIN;
-- Grant usage to runtime role
GRANT USAGE ON INTEGRATION quickstart_kafka_connector_access TO ROLE QUICKSTART_ROLE;

In [ ]:
DESCRIBE EXTERNAL ACCESS INTEGRATION quickstart_kafka_connector_access;